# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/cities.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["City_ID"])
weather_data_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

290

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
7,opuwo,72.06,5.44,0
70,ponta do sol,73.68,5.32,0
88,cidreira,76.56,5.50,0
138,morondava,79.08,9.06,0
139,vaini,78.90,1.59,0
176,nador,75.20,6.93,0
297,mogok,72.24,2.77,0
444,birjand,71.60,9.17,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: mahebourg.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: qaanaaq.
Missing field/result... skipping.
------------
Retrieving Results for Index 3: zhuhai.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: cape town.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: atuona.
Missing field/result... skipping.
------------
Retrieving Results for Index 6: flinders.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: opuwo.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: tiksi.
Missing field/result... skipping.
------------
Retrieving Results for Index 9: pushkinskiye gory.
Missing field/result... skipping.
------------
Retrieving Results for Index 10: aykhal.
Missing field/result... skipping.
------------
Retrieving Results fo

Retrieving Results for Index 91: bella vista.
Missing field/result... skipping.
------------
Retrieving Results for Index 92: xichang.
Missing field/result... skipping.
------------
Retrieving Results for Index 93: antioch.
Missing field/result... skipping.
------------
Retrieving Results for Index 94: najran.
Missing field/result... skipping.
------------
Retrieving Results for Index 95: agirish.
Missing field/result... skipping.
------------
Retrieving Results for Index 96: mataura.
Missing field/result... skipping.
------------
Retrieving Results for Index 97: katsuura.
Missing field/result... skipping.
------------
Retrieving Results for Index 98: shalinskoye.
Missing field/result... skipping.
------------
Retrieving Results for Index 99: barrow.
Missing field/result... skipping.
------------
Retrieving Results for Index 100: sorong.
Missing field/result... skipping.
------------
Retrieving Results for Index 101: kingston.
Missing field/result... skipping.
------------
Retrieving R

Retrieving Results for Index 181: tasiilaq.
Missing field/result... skipping.
------------
Retrieving Results for Index 182: udachnyy.
Missing field/result... skipping.
------------
Retrieving Results for Index 183: monrovia.
Missing field/result... skipping.
------------
Retrieving Results for Index 184: burnie.
Missing field/result... skipping.
------------
Retrieving Results for Index 185: kaitangata.
Missing field/result... skipping.
------------
Retrieving Results for Index 186: yamada.
Missing field/result... skipping.
------------
Retrieving Results for Index 187: mackay.
Missing field/result... skipping.
------------
Retrieving Results for Index 188: lagoa.
Missing field/result... skipping.
------------
Retrieving Results for Index 189: bluff.
Missing field/result... skipping.
------------
Retrieving Results for Index 190: saint-georges.
Missing field/result... skipping.
------------
Retrieving Results for Index 191: naryan-mar.
Missing field/result... skipping.
------------
Re

Retrieving Results for Index 271: coihaique.
Missing field/result... skipping.
------------
Retrieving Results for Index 272: sao felix do xingu.
Missing field/result... skipping.
------------
Retrieving Results for Index 273: santa rosa.
Missing field/result... skipping.
------------
Retrieving Results for Index 274: goderich.
Missing field/result... skipping.
------------
Retrieving Results for Index 275: vila velha.
Missing field/result... skipping.
------------
Retrieving Results for Index 276: rabo de peixe.
Missing field/result... skipping.
------------
Retrieving Results for Index 277: soyo.
Missing field/result... skipping.
------------
Retrieving Results for Index 278: cayenne.
Missing field/result... skipping.
------------
Retrieving Results for Index 279: saldanha.
Missing field/result... skipping.
------------
Retrieving Results for Index 280: santa ines.
Missing field/result... skipping.
------------
Retrieving Results for Index 281: hasaki.
Missing field/result... skippin

Retrieving Results for Index 362: datong.
Missing field/result... skipping.
------------
Retrieving Results for Index 363: vila franca do campo.
Missing field/result... skipping.
------------
Retrieving Results for Index 364: akureyri.
Missing field/result... skipping.
------------
Retrieving Results for Index 365: ulladulla.
Missing field/result... skipping.
------------
Retrieving Results for Index 366: ust-barguzin.
Missing field/result... skipping.
------------
Retrieving Results for Index 367: pampierstad.
Missing field/result... skipping.
------------
Retrieving Results for Index 368: zyryanovsk.
Missing field/result... skipping.
------------
Retrieving Results for Index 369: cooma.
Missing field/result... skipping.
------------
Retrieving Results for Index 370: sfantu gheorghe.
Missing field/result... skipping.
------------
Retrieving Results for Index 371: carballo.
Missing field/result... skipping.
------------
Retrieving Results for Index 372: shimoda.
Missing field/result...

Retrieving Results for Index 452: synya.
Missing field/result... skipping.
------------
Retrieving Results for Index 453: thanatpin.
Missing field/result... skipping.
------------
Retrieving Results for Index 454: altagracia de orituco.
Missing field/result... skipping.
------------
Retrieving Results for Index 455: anqing.
Missing field/result... skipping.
------------
Retrieving Results for Index 456: nome.
Missing field/result... skipping.
------------
Retrieving Results for Index 457: lahaina.
Missing field/result... skipping.
------------
Retrieving Results for Index 458: hereford.
Missing field/result... skipping.
------------
Retrieving Results for Index 459: san ramon de la nueva oran.
Missing field/result... skipping.
------------
Retrieving Results for Index 460: manuk mangkaw.
Missing field/result... skipping.
------------
Retrieving Results for Index 461: prainha.
Missing field/result... skipping.
------------
Retrieving Results for Index 462: pacific grove.
Missing field/r

Retrieving Results for Index 542: perth.
Missing field/result... skipping.
------------
Retrieving Results for Index 543: yabrud.
Missing field/result... skipping.
------------
Retrieving Results for Index 544: paraiso.
Missing field/result... skipping.
------------
Retrieving Results for Index 545: veraval.
Missing field/result... skipping.
------------
Retrieving Results for Index 546: novyy urgal.
Missing field/result... skipping.
------------
Retrieving Results for Index 547: tongren.
Missing field/result... skipping.
------------
-------End of Search-------


In [ ]:
# Display the hotel dataframe
print hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [ ]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig